In [2]:
import os
import pandas as pd
from datetime import date, datetime, timedelta
import yfinance as yf
import numpy as np
from pathlib import Path
from library import FileLoader

In [75]:
market_type_df = pd.read_pickle(r'Y:\因子回測_江建彰\上市上櫃歷年列表.pkl')
market_type_df.head(3)

ticker,1101,1102,1103,1104,1108,1109,1110,1201,1203,1210,...,9944,9945,9946,9949,9950,9951,9955,9958,9960,9962
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-05,TWSE,TWSE,TWSE,TWSE,TWSE,TWSE,TWSE,TWSE,TWSE,TWSE,...,TWSE,TWSE,TWSE,OTC,OTC,OTC,TWSE,TWSE,OTC,OTC
2015-01-06,TWSE,TWSE,TWSE,TWSE,TWSE,TWSE,TWSE,TWSE,TWSE,TWSE,...,TWSE,TWSE,TWSE,OTC,OTC,OTC,TWSE,TWSE,OTC,OTC
2015-01-07,TWSE,TWSE,TWSE,TWSE,TWSE,TWSE,TWSE,TWSE,TWSE,TWSE,...,TWSE,TWSE,TWSE,OTC,OTC,OTC,TWSE,TWSE,OTC,OTC
2015-01-08,TWSE,TWSE,TWSE,TWSE,TWSE,TWSE,TWSE,TWSE,TWSE,TWSE,...,TWSE,TWSE,TWSE,OTC,OTC,OTC,TWSE,TWSE,OTC,OTC
2015-01-09,TWSE,TWSE,TWSE,TWSE,TWSE,TWSE,TWSE,TWSE,TWSE,TWSE,...,TWSE,TWSE,TWSE,OTC,OTC,OTC,TWSE,TWSE,OTC,OTC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-15,TWSE,TWSE,TWSE,TWSE,TWSE,TWSE,TWSE,TWSE,TWSE,TWSE,...,TWSE,TWSE,TWSE,OTC,OTC,OTC,TWSE,TWSE,OTC,OTC
2025-04-16,TWSE,TWSE,TWSE,TWSE,TWSE,TWSE,TWSE,TWSE,TWSE,TWSE,...,TWSE,TWSE,TWSE,OTC,OTC,OTC,TWSE,TWSE,OTC,OTC
2025-04-17,TWSE,TWSE,TWSE,TWSE,TWSE,TWSE,TWSE,TWSE,TWSE,TWSE,...,TWSE,TWSE,TWSE,OTC,OTC,OTC,TWSE,TWSE,OTC,OTC


In [71]:
daily_adj_prices = FileLoader.load(r'Y:\因子回測_江建彰\daily_adj_prices.ftr')
daily_adj_prices.head(3)

⚡ 快取使用: Y:\因子回測_江建彰\daily_adj_prices.ftr


,trade_date,symbol_id,open_price,high_price,low_price,close_price,volume,amount,transaction,pe_ratio,...,pb_ratio,adj,adj_pe,adj_open_price,adj_high_price,adj_low_price,adj_close_price,adj_pe_ratio,adj_dividend_yield,adj_pb_ratio
0,2009-12-21,1558,77.9,77.9,76.3,76.4,113328,8691858,82,8.80,...,2.53,1.0,NaN,77.9,77.9,76.3,76.4,8.80,9.0,2.53
1,2009-12-22,1558,77.4,77.4,76.3,76.7,324931,24906456,163,8.84,...,2.54,1.0,NaN,77.4,77.4,76.3,76.7,8.84,9.0,2.54
2,2009-12-23,1558,76.8,79.6,76.6,78.5,712089,55651449,374,9.04,...,2.60,1.0,NaN,76.8,79.6,76.6,78.5,9.04,9.0,2.60


In [86]:
class OTC:
    daily_adj_prices = FileLoader.load(r'Y:\因子回測_江建彰\daily_adj_prices.ftr')
    market_type_df = pd.read_pickle(r'Y:\因子回測_江建彰\上市上櫃歷年列表.pkl')
    @staticmethod
    def amount(curr_date):
        all_tickers = OTC.market_type_df.columns
        Type_list = OTC.market_type_df.loc[curr_date]
        OTC_stock_list = [ticker 
                          for ticker, Type in zip(all_tickers, Type_list)
                          if Type=='OTC']
        
        cond1 = OTC.daily_adj_prices['trade_date']==curr_date
        cond2 = OTC.daily_adj_prices['symbol_id'].isin(OTC_stock_list)
        OTC_df = OTC.daily_adj_prices[cond1&cond2]
        return OTC_df['amount'].dropna().sum()



⚡ 快取使用: Y:\因子回測_江建彰\daily_adj_prices.ftr


In [87]:
amount_list = []
OTC_index_series = pd.read_pickle(r'Y:\因子回測_江建彰\櫃買報酬指數.pkl')
count = 0
for curr_date in OTC_index_series.index:
    if((count%100)==0): 
        print(curr_date)
    count+=1
    
    amount_list.append(OTC.amount(curr_date))
    
final_df = pd.DataFrame({'price' : OTC_index_series.values,
                         'amount' : amount_list},
                         index = OTC_index_series.index)

2015-01-05 00:00:00
2015-06-08 00:00:00
2015-11-02 00:00:00
2016-03-31 00:00:00
2016-08-25 00:00:00
2017-01-19 00:00:00
2017-06-23 00:00:00
2017-11-14 00:00:00
2018-04-17 00:00:00
2018-09-06 00:00:00
2019-01-29 00:00:00
2019-07-05 00:00:00
2019-11-29 00:00:00
2020-05-05 00:00:00
2020-09-24 00:00:00
2021-02-26 00:00:00
2021-07-23 00:00:00
2021-12-15 00:00:00
2022-05-20 00:00:00
2022-10-12 00:00:00
2023-03-16 00:00:00
2023-08-14 00:00:00
2024-01-05 00:00:00
2024-06-11 00:00:00
2024-11-07 00:00:00
2025-04-11 00:00:00


In [94]:
final_df[:-3]

,price,amount
Date,,
2015-01-05,182.84,18252821076
2015-01-06,179.74,21617016991
2015-01-07,179.70,18581502158
2015-01-08,180.34,22408731504
2015-01-09,180.66,20682092591
...,...,...
2025-04-10,359.32,37524464732
2025-04-11,364.56,80100106455
2025-04-14,371.10,76806780280


In [95]:
#取[:-3]是因為後面三筆資料沒有紀錄，所以成交量是零
final_df[:-3].to_feather('櫃買報酬指數及成交金額.ftr')

In [97]:
FileLoader.load('櫃買報酬指數及成交金額.ftr')

📂 讀取: 櫃買報酬指數及成交金額.ftr


,price,amount
Date,,
2015-01-05,182.84,18252821076
2015-01-06,179.74,21617016991
2015-01-07,179.70,18581502158
2015-01-08,180.34,22408731504
2015-01-09,180.66,20682092591
...,...,...
2025-04-10,359.32,37524464732
2025-04-11,364.56,80100106455
2025-04-14,371.10,76806780280


# 檢查區

In [93]:
final_df

,price,amount
Date,,
2015-01-05,182.84,18252821076
2015-01-06,179.74,21617016991
2015-01-07,179.70,18581502158
2015-01-08,180.34,22408731504
2015-01-09,180.66,20682092591
...,...,...
2025-04-15,385.09,68807135388
2025-04-16,378.69,81137676824
2025-04-17,377.57,0


In [89]:
OTC_index_series

Date
2015-01-05    182.84
2015-01-06    179.74
2015-01-07    179.70
2015-01-08    180.34
2015-01-09    180.66
               ...  
2025-04-15    385.09
2025-04-16    378.69
2025-04-17    377.57
2025-04-18    377.69
2025-04-21    367.16
Name: IR0043, Length: 2507, dtype: float64

In [91]:
print(OTC.amount('2015-01-08'))
print(OTC.amount('2025-04-16'))

22408731504
81137676824
